In [ ]:
import pandas as pd 
import numpy as np
import pickle
import xgboost as xgb
from sklearn.metrics import mean_absolute_error

## Predict for the given datapoint

In [ ]:
def function1(input):
    
    shift = 200 

    # adding extra features for data point 
    input['is_cat3_sameas_cat90']=np.where(input['cat3']==input['cat90'],1,0)

    input['is_cat7_sameas_cat89']=np.where(input['cat7']==input['cat89'],1,0)

    input['is_cat2_sameas_cat9']=np.where(input['cat2']==input['cat9'],1,0)

    input['is_cat6_sameas_cat50']=np.where(input['cat6']==input['cat50'],1,0)

    COMB_FEATURE = 'cat80,cat87,cat57,cat12,cat79,cat10,cat7,cat89,cat2,cat72,' \
            'cat81,cat11,cat1,cat13,cat9,cat3,cat16,cat90,cat23,cat36,' \
            'cat73,cat103,cat40,cat28,cat111,cat6,cat76,cat50,cat5,' \
            'cat4,cat14,cat38,cat24,cat82,cat25'.split(',')

    cat_features = [i for i in input.columns if i[:3]=='cat']

    def encode(charcode):
        '''This function simply computes the ordinal value with some modifications'''
        r = 0
        ln = len(str(charcode))
        for i in range(ln):
            r += (ord(str(charcode)[i]) - ord('A') + 1) * 26 ** (ln - i - 1)
        return r

    for i in range(len(COMB_FEATURE)):
        for j in range(i+1,len(COMB_FEATURE)):
            feature = COMB_FEATURE[i]+'_'+COMB_FEATURE[j]
            input[feature]=input[COMB_FEATURE[i]]+input[COMB_FEATURE[j]]
            input[feature]=input[feature].apply(encode)

    for i in cat_features:
        input[i]=input[i].apply(encode)

    dpoint = xgb.DMatrix(input)

    loaded_model = pickle.load(open('/content/drive/My Drive/finalized_model_lexical_coding.pkl', 'rb'))

    result = loaded_model.predict(dpoint)
    result = np.exp(result)-shift
    
    return result[0]

In [ ]:
test_point  = pd.read_csv('/content/drive/My Drive/allstate-claims-severity/test.csv',nrows=1)

test_point = test_point.drop(['id'],axis=1)

print(function1(test_point))

2300.935


## Predict the loss for given datapoints

In [ ]:
def function2(input,actual_value):
    
    shift = 200 

    # adding extra features for data point 
    input['is_cat3_sameas_cat90']=np.where(input['cat3']==input['cat90'],1,0)

    input['is_cat7_sameas_cat89']=np.where(input['cat7']==input['cat89'],1,0)

    input['is_cat2_sameas_cat9']=np.where(input['cat2']==input['cat9'],1,0)

    input['is_cat6_sameas_cat50']=np.where(input['cat6']==input['cat50'],1,0)

    COMB_FEATURE = 'cat80,cat87,cat57,cat12,cat79,cat10,cat7,cat89,cat2,cat72,' \
            'cat81,cat11,cat1,cat13,cat9,cat3,cat16,cat90,cat23,cat36,' \
            'cat73,cat103,cat40,cat28,cat111,cat6,cat76,cat50,cat5,' \
            'cat4,cat14,cat38,cat24,cat82,cat25'.split(',')

    cat_features = [i for i in input.columns if i[:3]=='cat']

    def encode(charcode):
        '''This function simply computes the ordinal value with some modifications'''
        r = 0
        ln = len(str(charcode))
        for i in range(ln):
            r += (ord(str(charcode)[i]) - ord('A') + 1) * 26 ** (ln - i - 1)
        return r

    for i in range(len(COMB_FEATURE)):
        for j in range(i+1,len(COMB_FEATURE)):
            feature = COMB_FEATURE[i]+'_'+COMB_FEATURE[j]
            input[feature]=input[COMB_FEATURE[i]]+input[COMB_FEATURE[j]]
            input[feature]=input[feature].apply(encode)

    for i in cat_features:
        input[i]=input[i].apply(encode)

    dpoint = xgb.DMatrix(input)

    loaded_model = pickle.load(open('/content/drive/My Drive/finalized_model_lexical_coding.pkl', 'rb'))

    result = loaded_model.predict(dpoint)
    result = np.exp(result)-shift

    error = mean_absolute_error(result,actual_value)

    return result,error

In [ ]:
datapoint = pd.read_csv('/content/drive/My Drive/allstate-claims-severity/train.csv',nrows=2)

actual_value = datapoint['loss']

datapoint = datapoint.drop(['id','loss'],axis=1)

predictions , mae  = function2(datapoint,actual_value)

print('Predictions of given data points are : \n {0}'.format(predictions))

print('Mean absolute error : {0}'.format(mae))

Predictions of given data points are : 
 [2389.5442 3338.2742]
Mean absolute error : 1115.5191796875001
